In [1]:
import os

from dotenv import load_dotenv

load_dotenv()
os.environ.setdefault(
    "USER_AGENT",
    "Mozilla/5.0 (X11; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0",
)

import re
from pprint import PrettyPrinter

from bs4 import BeautifulSoup
from httpx import Client

In [4]:
# url = 'https://www.douglas.de/de/c/gesicht/gesichtsmasken/feuchtigkeitsmasken/120308'
# url = 'https://www.douglas.de/de/p/3001055831'  # Avocado
url = "https://www.douglas.de/de/p/3000037624"  # with discount

In [5]:
client = Client()
res = client.get(
    url,
    headers={
        "Accept": "text/html",
        "User-Agent": os.environ.get("USER_AGENT"),
    },
)

In [6]:
soup = BeautifulSoup(res.text, features="lxml")

In [11]:
PrettyPrinter(indent=2).pprint(
    {
        "name": soup.find("span", attrs={"class": "header-name"}).text,
        "image": soup.find("img", attrs={"class": "image swiper-lazy"}).get(
            "data-lazy-src"
        ),
        "variants": [
            {
                "name": sou.find(
                    "div", attrs={"class": "product-detail__variant-name"}
                ).text,
                "price": re.search(
                    r"\d+\.\d+",
                    sou.find("span", attrs={"class": "product-price__price"})
                    .text.replace("\xa0", " ")
                    .replace(",", "."),
                    flags=re.I,
                ).group(),
            }
            for sou in soup.find_all(
                "div", attrs={"class": "product-detail__variant-row"}
            )
        ],
        "product_labels": [
            s.text
            for s in soup.find("div", attrs={"class": "product-labels"}).find_all(
                "span", {"class": "product-label__name"}
            )
        ],
        "product_details": {
            sou.find_all("span")[0].text: sou.find_all("span")[1].text
            for sou in soup.find(
                "div", attrs={"data-testid": "product-detail-info__classifications"}
            ).find_all("div")
        },
        "description": soup.find(
            "div", attrs={"class": "truncate product-details__description"}
        ).text,
        "avg_rating": re.search(
            r"^\d+\.\d+",
            soup.find("span", attrs={"class": "ratings-info"}).text.replace(
                "\xa0", " "
            ),
            flags=re.I,
        ).group(),
        "total_ratings": re.search(
            r"\(\d+\)$",
            soup.find("span", attrs={"class": "ratings-info"}).text.replace(
                "\xa0", " "
            ),
            flags=re.I,
        )
        .group()
        .strip("()"),
    }
)

{ 'avg_rating': '4.4',
  'description': 'Für eine strahlende & makellose HautMit der Rare Earth Pore '
                 "Cleansing Masque präsentiert Kiehl's eine effiziente, "
                 'porentief klärende Maske auf Basis von seltener weißer '
                 'Amazonas-Tonerde aus fairem Handel. Sie befreit die Haut von '
                 'überschüssigem Sebum und gleicht Partien aus, die zu '
                 'Unreinheiten neigen. Diese klärenden Wirkstoffe werden '
                 'kombiniert mit pflegenden Inhaltsstoffen wie '
                 'hydratisierender Aloe vera und beruhigendem Hafermehl. Das '
                 'Ergebnis ist eine porentief gereinigte Haut, die von '
                 'Mitessern, Pickeln und Rötungen befreit ist.\xa0',
  'image': 'https://media.douglas.de/medias/edrQ9J939174-0-dgl-DE.jpg?context=bWFzdGVyfGltYWdlc3wxMTA3MDl8aW1hZ2UvanBlZ3xhR0l3TDJoaVlTODRPREEyTXpreU1EY3dNVGMwTDJWa2NsRTVTamt6T1RFM05GOHdYMlJuYkMxRVJTNXFjR2N8YzU2OWIzMzkyOTM2NDQxNWQzM2V